In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [55]:
con=sqlite3.connect("tvmaze.sqlite")
df_info=pd.read_sql_query("SELECT * from tvmaze",con)
df_genre=pd.read_sql_query("SELECT * from tvmaze_genre",con)
df_casting=pd.read_sql_query("SELECT * from tvmaze_casting",con)

In [56]:

df_genre["genre"]
df_info["description"].dropna(axis=0)
df_info['description'].fillna('', inplace=True)
len(df_genre["genre"].unique())

28

In [64]:
def remove_tags(text):
    text = text.replace('<p>', '')
    text = text.replace('</p>', '')
    text = text.replace('</b>', '')
    text = text.replace('<b>', '')
    text = text.replace(r'\W', '')
    text=text.lower()
    return text

In [68]:
df_info["description"]=df_info["description"].apply(lambda x: remove_tags(x))
df_info["description"].str.lower()

0        under the dome is the story of a small town th...
1        you are being watched. the government has a se...
2        based on the critically acclaimed series of no...
3        after a violent shipwreck, billionaire playboy...
4        touch darkness and darkness touches you back. ...
                               ...                        
65983    a cooperative drama between a fraudster who ca...
65984                                                     
65985    chen jun, a pseudo-otaku who aspires to eat an...
65986    a professional killer decides to retract from ...
65987    this gripping three-part series explores one o...
Name: description, Length: 65988, dtype: object

In [69]:
id_list=df_info["tvmaze_id"].to_list()

df_genre["genre"]
df_merged=pd.merge(df_info,df_genre,on="tvmaze_id",how="inner")
df_merged.to_csv("datafillter.csv")

In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Load data
data = pd.read_csv('datafillter.csv')
data=data.dropna()

# Preprocessing

X = data['description']
y = data['genre']

# Tokenization
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=200)

# Encoding labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build model
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(28, activation='softmax'))  # Adjust the output dimension based on your number of genres
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=64)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: %.2f" % (accuracy*100))


Epoch 1/10
136/621 [=====>........................] - ETA: 8:21 - loss: 2.8019 - accuracy: 0.2134

In [35]:
import pandas as pd

# Giả sử df là DataFrame của bạn
# df = pd.DataFrame(your_data)

# Tạo một DataFrame mẫu
data = {'Text': ['<p>This is an example</p>', 'Some text with </b> tag', 'Another <p> example </b>']}
df = pd.DataFrame(data)

# Loại bỏ các chuỗi <p> và </b> từ cột 'Text'
df['Text'] = df['Text'].str.replace('<p>', '').str.replace('</b>', '')

# In DataFrame sau khi loại bỏ các chuỗi
print(df)

                     Text
0  This is an example</p>
1     Some text with  tag
2       Another  example 


In [2]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]
print(tokenized_corpus)
bm25 = BM25Okapi(tokenized_corpus)

[['Hello', 'there', 'good', 'man!'], ['It', 'is', 'quite', 'windy', 'in', 'London'], ['How', 'is', 'the', 'weather', 'today?']]
